In [104]:
from pyopenms import *

example_filename= "Epemicins.mzML"
exp = MSExperiment()
MzMLFile().load(example_filename, exp)
exp.sortSpectra(True)

In [105]:
# Run mass trace detection
mass_traces = []
mtd = MassTraceDetection()
mtd_par = mtd.getDefaults()
mtd_par.setValue("mass_error_ppm", 10.0) 
mtd_par.setValue("noise_threshold_int", 1.0e04)
mtd_par.setValue("chrom_peak_snr", 3.0)
mtd_par.setValue("common:chrom_fwhm", 1.5)
mtd_par.setValue("min_trace_length", 3.0)
mtd_par.setValue("max_trace_length", 60.0)
mtd.setParameters(mtd_par)
mtd.run(exp, mass_traces, 0)

In [123]:
mass_traces_split = []
mass_traces_final = []
epd = ElutionPeakDetection()
epd_par = epd.getDefaults()
epd_par.setValue("width_filtering", "fixed")
epd_par.setValue("chrom_fwhm", 1.5)
epd_par.setValue("min_fwhm", 1.0)
epd_par.setValue("max_fwhm", 30.0)
epd.setParameters(epd_par)
epd.detectPeaks(mass_traces, mass_traces_split)
epd.filterByPeakWidth(mass_traces_split, mass_traces_final)

In [124]:
# Run feature detection
feature_map_FFM = FeatureMap()
feat_chrom = []
ffm = FeatureFindingMetabo()
ffm_par = ffm.getDefaults() 
ffm_par.setValue("isotope_filtering_model", "none")
ffm_par.setValue("remove_single_traces", "false")
ffm_par.setValue("mz_scoring_by_elements", "false")
ffm_par.setValue("report_convex_hulls", "true")
ffm.setParameters(ffm_par)
ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
feature_map_FFM.setUniqueIds()
fh = FeatureXMLFile()
fh.store('./EpemicinsFeatureFindingMetabo.featureXML', feature_map_FFM)

In [125]:
# Run metabolite adduct decharging detection
# With SIRIUS you are only able to use singly charged adducts
mfd = MetaboliteFeatureDeconvolution()
mdf_par = mfd.getDefaults()
mdf_par.setValue("potential_adducts",  [b"H:+:0.6",b"Na:+:0.2",b"NH4:+:0.1", b"H2O:-:0.1"])
mdf_par.setValue("charge_min", 1, "Minimal possible charge")
mdf_par.setValue("charge_max", 1, "Maximal possible charge")
mdf_par.setValue("charge_span_max", 1)
mdf_par.setValue("max_neutrals", 1)
mfd.setParameters(mdf_par)
feature_map_DEC = FeatureMap()
cons_map0 = ConsensusMap()
cons_map1 = ConsensusMap()
mfd.compute(feature_map_FFM, feature_map_DEC, cons_map0, cons_map1)
fxml = FeatureXMLFile()
fh.store('./EpemicinsMFD.featureXML', feature_map_FFM)

In [128]:
from pandas import DataFrame
import pandas as pd
from pyteomics.openms import featurexml

with featurexml.read('./EpemicinsMFD.featureXML') as f:
    features_list = [FXML for FXML in f]
    
df = pd.DataFrame() 
for feat in features_list:
    idx = feat['id']
    for key in feat.keys():
        if key == 'id':
           pass
        elif key == 'position':
            pos_list = feat['position']
            for pos in pos_list:
                if pos['dim'] == '0':
                    df.loc[idx, 'RT'] = pos['position']
                elif pos['dim'] == '1':
                    df.loc[idx, 'mz'] = pos['position']
        elif key == 'intensity':
            int_list = feat['intensity']
            df.loc[idx, 'intensity'] = feat['intensity']
        elif key == 'charge':
            charge_list = feat['charge']
            df.loc[idx, 'charge'] = feat['charge']
        elif key == 'FWHM':
            FWHM_list = feat['FWHM']
            df.loc[idx, 'FWHM'] = feat['FWHM']
        elif key == 'max_height':
            max_height_list = feat['max_height']
            df.loc[idx, 'max_height'] = feat['max_height']
        elif key == 'num_of_masstraces':
            num_of_masstraces_list = feat['num_of_masstraces']
            df.loc[idx, 'num_of_masstraces'] = feat['num_of_masstraces']
        elif key == 'masstrace_intesity':
            masstrace_intesity_list = feat['masstrace_intesity']
            df.loc[idx, 'masstrace_intesity'] = feat['masstrace_intesity']
        elif key == 'masstrace_centroid_rt':
            masstrace_intesity_list = feat['masstrace_centroid_rt']
            df.loc[idx, 'masstrace_centroid_rt'] = feat['masstrace_centroid_rt']
        elif key == 'masstrace_centroid_mz':
            masstrace_intesity_list = feat['masstrace_centroid_mz']
            df.loc[idx, 'masstrace_centroid_mz'] = feat['masstrace_centroid_mz']
        elif key == 'isotope_distances':
            masstrace_intesity_list = feat['isotope_distances']
            df.loc[idx, 'isotope_distances'] = feat['isotope_distances']
        elif key == 'dc_charge_adducts':
            dc_charge_adducts_list = feat['dc_charge_adducts']
            df.loc[idx, 'dc_charge_adducts'] = feat['dc_charge_adducts']

In [129]:
df

,RT,mz,intensity,charge,FWHM,max_height,num_of_masstraces,masstrace_centroid_rt,masstrace_centroid_mz,isotope_distances
f_15530212072324173342,235.216210063999796,197.128348894490102,1.445694e05,0,2.496554,7.473009e+04,1.0,[235.216210063999796],[197.128348894490102],[]
f_15652756236035929738,223.754813167999799,199.144060285793216,1.779977e05,0,1.900442,8.749283e+04,1.0,[223.754813167999799],[199.144060285793216],[]
f_6108469367494092383,265.115979007999783,199.14407972525936,1.667679e05,0,2.038713,7.590222e+04,1.0,[265.115979007999783],[199.14407972525936],[]
f_13038426927472566551,239.061554145000002,201.123327651824951,1.61971e05,0,2.265037,7.972224e+04,1.0,[239.061554145000002],[201.123327651824951],[]
f_171643581759886120,330.529096272000004,201.123355420635306,2.090583e05,0,1.700533,1.361184e+05,1.0,[330.529096272000004],[201.123355420635306],[]
...,...,...,...,...,...,...,...,...,...,...
f_10343101875387285161,106.297279615999798,1775.552747330655166,1.916161e06,0,5.788356,4.517596e+05,1.0,[106.297279615999798],[1775.552747330655166],[]
f_4860832575957268400,105.360296063999996,1776.05419662686495,2.256658e06,2,3.018481,1.022467e+06,5.0,"[105.360296063999996, 105.360296063999996, 105...","[1776.05419662686495, 1776.55614417148081, 177...","[0.501947544615859, 0.501232335921031, 0.50231..."
f_5061152849278334852,106.297279615999798,1776.221340962091062,5.972576e05,0,4.593563,1.570660e+05,1.0,[106.297279615999798],[1776.221340962091062],[]
f_11085486815860129068,107.215393951999801,1776.889239898031747,4.857327e05,0,2.774759,1.886686e+05,1.0,[107.215393951999801],[1776.889239898031747],[]


In [130]:
df.to_csv("EpemicinsFFM.csv")